In [13]:
! pip install --user --quiet google-cloud-aiplatform==1.25.0

# Install pandas
! pip install --user --quiet pandas

# Install HuggingFace Datasets
! pip install --user --quiet datasets

# Install Python client for Google Search API
! pip install --user --quiet google-api-python-client

# PDF loader
! pip install --user --quiet pypdf


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
# Below libraries are required to build a SQL engine for BigQuery
!pip install --user SQLAlchemy==1.4.48 --quiet
!pip install --user sqlalchemy-bigquery --quiet

In [3]:
!pip show langchain version

Name: langchain
Version: 0.0.229
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
import sqlalchemy
import google.cloud.bigquery

print(f"SQLAlchemy version: {sqlalchemy.__version__}")
print(f"BigQuery version: {google.cloud.bigquery.__version__}")


SQLAlchemy version: 1.4.48
BigQuery version: 3.25.0


In [4]:
from google.cloud import aiplatform
PROJECT_ID = "sql-querygenerator"  # Update this with your project id
LOCATION = "us-central1"

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [5]:
import langchain
print(f"LangChain version: {langchain.__version__}")

from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")

# Initialize Vertex AI SDK
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

LangChain version: 0.0.229
Vertex AI SDK version: 1.59.0


In [6]:
import time
from typing import Any, Mapping, List, Dict, Optional, Tuple, Union
from dataclasses import dataclass, field

from pydantic import BaseModel, Extra, root_validator

from langchain.llms.base import LLM
from langchain.embeddings.base import Embeddings
from langchain.chat_models.base import BaseChatModel
from langchain.llms.utils import enforce_stop_tokens
from langchain.schema import Generation, LLMResult
from langchain.schema import AIMessage, BaseMessage, ChatGeneration, ChatResult, HumanMessage, SystemMessage

from vertexai.preview.language_models import TextGenerationResponse, ChatSession

def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print('Waiting')
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print('.', end='')
            time.sleep(sleep_time)

class _VertexCommon(BaseModel):
    client: Any = None
    model_name: str = "text-bison@001"
    temperature: float = 0.2
    top_p: int = 0.8
    top_k: int = 40
    max_output_tokens: int = 200

    @property
    def _default_params(self) -> Mapping[str, Any]:
        return {
            "temperature": self.temperature,
            "top_p": self.top_p,
            "top_k": self.top_k,
            "max_output_tokens": self.max_output_tokens
        }

    def _predict(self, prompt: str, stop: Optional[List[str]]) -> str:
        res = self.client.predict(prompt, **self._default_params)
        return self._enforce_stop_words(res.text, stop)

    def _enforce_stop_words(self, text: str, stop: Optional[List[str]]) -> str:
        if stop:
            return enforce_stop_tokens(text, stop)
        return text

    @property
    def _llm_type(self) -> str:
        return "vertex_ai"

class VertexLLM(_VertexCommon, LLM):
    model_name: str = "text-bison@001"

    @root_validator(allow_reuse=True)
    def validate_environment(cls, values: Dict) -> Dict:
        try:
            from vertexai.preview.language_models import TextGenerationModel
        except ImportError:
            raise ValueError("Could not import Vertex AI LLM python package.")
        try:
            values["client"] = TextGenerationModel.from_pretrained(values["model_name"])
        except AttributeError:
            raise ValueError("Could not set Vertex Text Model client.")
        return values

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return self._predict(prompt, stop)

@dataclass
class _MessagePair:
    question: HumanMessage
    answer: AIMessage

@dataclass
class _ChatHistory:
    history: List[_MessagePair] = field(default_factory=list)
    system_message: Optional[SystemMessage] = None

def _parse_chat_history(history: List[BaseMessage]) -> _ChatHistory:
    if not history:
        return _ChatHistory()
    first_message = history[0]
    system_message = first_message if isinstance(first_message, SystemMessage) else None
    chat_history = _ChatHistory(system_message=system_message)
    messages_left = history[1:] if system_message else history
    for question, answer in zip(messages_left[::2], messages_left[1::2]):
        if not isinstance(question, HumanMessage) or not isinstance(answer, AIMessage):
            raise ValueError("A human message should follow a bot one.")
        chat_history.history.append(_MessagePair(question=question, answer=answer))
    return chat_history

class _VertexChatCommon(_VertexCommon):
    model_name: str = "chat-bison@001"

    @root_validator(allow_reuse=True)
    def validate_environment(cls, values: Dict) -> Dict:
        try:
            from vertexai.preview.language_models import ChatModel
        except ImportError:
            raise ValueError("Could not import Vertex AI LLM python package.")
        try:
            values["client"] = ChatModel.from_pretrained(values["model_name"])
        except AttributeError:
            raise ValueError("Could not set Vertex Text Model client.")
        return values

    def _response_to_chat_results(self, response: TextGenerationResponse, stop: Optional[List[str]]) -> ChatResult:
        text = self._enforce_stop_words(response.text, stop)
        return ChatResult(generations=[ChatGeneration(message=AIMessage(content=text))])

class VertexChat(_VertexChatCommon, BaseChatModel):
    model_name: str = "chat-bison@001"
    chat: Any = None

    def send_message(self, message: Union[HumanMessage, str], stop: Optional[List[str]] = None) -> ChatResult:
        text = message.content if isinstance(message, BaseMessage) else message
        response = self.chat.send_message(text)
        text = self._enforce_stop_words(response.text, stop)
        return ChatResult(generations=[ChatGeneration(message=AIMessage(content=text))])

    def _generate(self, messages: List[BaseMessage], stop: Optional[List[str]]) -> ChatResult:
        if not messages:
            raise ValueError("You should provide at least one message to start the chat!")
        question = messages[-1]
        if not isinstance(question, HumanMessage):
            raise ValueError(f"Last message in the list should be from human, got {question.type}.")
        self.start_chat(messages[:-1])
        return self.send_message(question)

    def start_chat(self, messages: List[BaseMessage]) -> None:
        history = _parse_chat_history(messages)
        context = history.system_message.content if history.system_message else None
        self.chat = self.client.start_chat(context=context, **self._default_params)
        for pair in history.history:
            self.chat._history.append((pair.question.content, pair.answer.content))

    def clear_chat(self) -> None:
        self.chat = None

    @property
    def history(self) -> List[BaseMessage]:
        history: List[BaseMessage] = []
        if self.chat:
            for question, answer in self.chat._history:
                history.append(HumanMessage(content=question))
                history.append(AIMessage(content=answer))
        return history

    async def _agenerate(self, messages: List[BaseMessage], stop: Optional[List[str]]) -> ChatResult:
        raise NotImplementedError("Vertex AI doesn't support async requests at the moment.")

class VertexMultiTurnChat(_VertexChatCommon, BaseChatModel):
    model_name: str = "chat-bison@001"
    chat: Optional[ChatSession] = None

    def clear_chat(self) -> None:
        self.chat = None

    def start_chat(self, message: Optional[SystemMessage] = None) -> None:
        if self.chat:
            raise ValueError("Chat has already been started. Please, clear it first.")
        if message and not isinstance(message, SystemMessage):
            raise ValueError("Context should be a system message")
        context = message.content if message else None
        self.chat = self.client.start_chat(context=context, **self._default_params)

    @property
    def history(self) -> List[Tuple[str]]:
        if self.chat:
            return self.chat._history
        return []

    def _generate(self, messages: List[BaseMessage], stop: Optional[List[str]]) -> ChatResult:
        if len(messages) != 1:
            raise ValueError("You should send exactly one message to the chat each turn.")
        if not self.chat:
            raise ValueError("You should start_chat first!")
        response = self.chat.send_message(messages[0].content)
        return self._response_to_chat_results(response, stop=stop)

    async def _agenerate(self, messages: List[BaseMessage], stop: Optional[List[str]]) -> ChatResult:
        raise NotImplementedError("Vertex AI doesn't support async requests at the moment.")

class VertexEmbeddings(Embeddings, BaseModel):
    model_name: str = "textembedding-gecko@001"
    model: Any
    requests_per_minute: int = 15

    @root_validator(allow_reuse=True)
    def validate_environment(cls, values: Dict) -> Dict:
        try:
            from vertexai.preview.language_models import TextEmbeddingModel
        except ImportError:
            raise ValueError("Could not import Vertex AI LLM python package.")
        try:
            values["model"] = TextEmbeddingModel
        except AttributeError:
            raise ValueError("Could not set Vertex Text Model client.")
        return values

    class Config:
        extra = Extra.forbid

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        self.model = self.model.from_pretrained(self.model_name)
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)
        while docs:
            head, docs = docs[:2], docs[2:]
            chunk = self.model.get_embeddings(head)
            results.extend(chunk)
            next(limiter)
        return [r.values for r in results]

    def embed_query(self, text: str) -> List[float]:
        single_result = self.embed_documents([text])
        return single_result[0]

In [7]:
REQUESTS_PER_MINUTE = 100

llm = VertexLLM(
    model_name='text-bison@001',
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

chat = VertexChat()

mchat = VertexMultiTurnChat(max_output_tokens=1024)

embedding = VertexEmbeddings(requests_per_minute=REQUESTS_PER_MINUTE)


In [8]:
from google.cloud import bigquery
from google.cloud.bigquery import Client

client = Client(project=PROJECT_ID)

# List datasets in the bigquery-public-data project
datasets = list(client.list_datasets(project="bigquery-public-data"))

dataset_names = [dataset.dataset_id for dataset in datasets]
print("Datasets in bigquery-public-data:")
print(dataset_names)

# Select a dataset (simulate user selection)
selected_dataset = dataset_names[0]  # For example, you can change this to any dataset from the list
print(f"Selected Dataset: {selected_dataset}")

Datasets in bigquery-public-data:
['america_health_rankings', 'austin_311', 'austin_bikeshare', 'austin_crime', 'austin_incidents', 'austin_waste', 'baseball', 'bbc_news', 'bigqueryml_ncaa', 'bitcoin_blockchain', 'blackhole_database', 'blockchain_analytics_ethereum_mainnet_us', 'bls', 'bls_qcew', 'breathe', 'broadstreet_adi', 'catalonian_mobile_coverage', 'catalonian_mobile_coverage_eu', 'census_bureau_acs', 'census_bureau_construction', 'census_bureau_international', 'census_bureau_usa', 'census_opportunity_atlas', 'census_utility', 'cfpb_complaints', 'chicago_crime', 'chicago_taxi_trips', 'clemson_dice', 'cloud_storage_geo_index', 'cms_codes', 'cms_medicare', 'cms_synthetic_patient_data_omop', 'country_codes', 'covid19_aha', 'covid19_covidtracking', 'covid19_ecdc', 'covid19_ecdc_eu', 'covid19_genome_sequence', 'covid19_geotab_mobility_impact', 'covid19_geotab_mobility_impact_eu', 'covid19_google_mobility', 'covid19_google_mobility_eu', 'covid19_govt_response', 'covid19_italy', 'covid

In [9]:
# List tables in the selected dataset
tables = list(client.list_tables(dataset=f"bigquery-public-data.{selected_dataset}"))
table_names = [table.table_id for table in tables]
print("Tables in selected dataset:")
print(table_names)

# Select a table (simulate user selection)
selected_table = table_names[0]  # For example, you can change this to any table from the list
print(f"Selected Table: {selected_table}")


Tables in selected dataset:
['ahr', 'america_health_rankings']
Selected Table: ahr


In [10]:
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# Define the source table
source_table_id = f"bigquery-public-data.america_health_rankings.ahr"

# Get the schema of the source table
table = client.get_table(source_table_id)
schema = table.schema

# Print the schema
for field in schema:
    print(f"Field: {field.name}, Type: {field.field_type}")


Field: edition, Type: INTEGER
Field: report_type, Type: STRING
Field: measure_name, Type: STRING
Field: state_name, Type: STRING
Field: subpopulation, Type: STRING
Field: value, Type: FLOAT
Field: lower_ci, Type: FLOAT
Field: upper_ci, Type: FLOAT
Field: source, Type: STRING
Field: source_date, Type: STRING


In [11]:
# Extract and print the column names
column_names = [field.name for field in schema]
print("Column names:", column_names)

Column names: ['edition', 'report_type', 'measure_name', 'state_name', 'subpopulation', 'value', 'lower_ci', 'upper_ci', 'source', 'source_date']


In [12]:
from google.cloud import bigquery
from google.cloud.bigquery import Client

client = Client(project=PROJECT_ID)
dataset_id = "america_health_rankings"

# Check if dataset exists
datasets = list(client.list_datasets())
dataset_exists = any(dataset.dataset_id == dataset_id for dataset in datasets)

if not dataset_exists:
    query = f"""
    CREATE SCHEMA `{PROJECT_ID}.{dataset_id}`
    OPTIONS(
      location="us"
    )
    """
    query_job = client.query(query)
    print(query_job.result())
else:
    print(f"Dataset {dataset_id} already exists.")

Dataset america_health_rankings already exists.


In [13]:
query = f"""
create or replace table `{PROJECT_ID}.{dataset_id}.ahr`
as
select 
edition,
report_type, 
measure_name, 
state_name,
subpopulation, 
value, 
lower_ci, 
upper_ci, 
source, 
source_date
from `bigquery-public-data.america_health_rankings.ahr`
""".format(
    PROJECT_ID=PROJECT_ID, dataset_id=dataset_id,
)
query_job = client.query(query)
print(query_job.result())

In [14]:
project_id = PROJECT_ID  # @param {type:"string"}
location = LOCATION  # @param {type:"string"}
dataset_id = 'america_health_rankings' # @param {type:"string"}
table_name = 'ahr' # @param {type:"string"}

In [15]:
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import pandas as pd

In [16]:
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}")

In [17]:
query=f"""SELECT * FROM `{project_id}.{dataset_id}.{table_name}`"""
engine.execute(query).first()

/var/tmp/ipykernel_131639/4098976475.py:2: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  engine.execute(query).first()


(2021, '2021 Health Disparities', 'Asthma', 'Vermont', 'Black/African American', None, None, None, 'CDC, Behavioral Risk Factor Surveillance System', '2017-2019')

In [18]:
from langchain import SQLDatabase, SQLDatabaseChain
from langchain.prompts.prompt import PromptTemplate

def bq_qna(question):
    # Create SQLDatabase instance from BQ engine
    db = SQLDatabase(engine=engine, metadata=MetaData(bind=engine), include_tables=[table_name])
    
    # Create SQL DB Chain with the initialized LLM and above SQLDB instance
    db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)

    # Define prompt for BigQuery SQL
    _googlesql_prompt = """You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Use the following format:
    Question: "Question here"
    SQLQuery: "SQL Query to run"
    SQLResult: "Result of the SQLQuery"
    Answer: "Final answer here"
    Only use the following tables:
    {table_info}

    If someone asks for a specific month, use ActivityDate between current month's start date and current month's end date

    Question: {input}"""

    BigQuerySQL_PROMPT = PromptTemplate(
        input_variables=["input", "table_info", "top_k"],
        template=_googlesql_prompt,
    )

    # Passing question to the prompt template
    final_prompt = BigQuerySQL_PROMPT.format(input=question, table_info=table_name, top_k=10000)

    # Pass final prompt to SQL Chain
    output = db_chain(final_prompt)

    return output['result'], output['intermediate_steps'][0]

# Testing the function
result, steps = bq_qna('Count total number of reports')
print("Result:", result)
print("Intermediate Steps:", steps)




> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Use the following format:
    Question: "Question here"
    SQLQuery: "SQL Query to run"
    SQLResult: "Result of the SQLQuery"
    Answer:

In [19]:
import streamlit as st

In [21]:
%%writefile stapp.py

import streamlit as st
st.title("Text to SQL on BigQuery Dataset with LangChain")

st.write("Ask a question about the dataset:")

question = st.text_input("Question")

if st.button("Get Answer"):
    if question:
        sql_query, results = bq_qna(question)
        st.write("Generated SQL Query:")
        st.code(sql_query)
        st.write("Query Results:")
        st.write(results)
    else:
        st.error("Please enter a question.")

Writing stapp.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com
! streamlit run stapp.py & npx localtunnel --port 8501

104.197.237.27
2024-07-22 02:16:01.150 INFO    numexpr.utils: NumExpr defaulting to 4 threads.
⠙⠹⠸⠼⠴⠦⠧your url is: https://dry-tigers-film.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://10.128.0.2:8501
  External URL: http://104.197.237.27:8501

2024-07-22 02:16:45.905 Uncaught app exception
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/streamlit/scriptrunner/script_runner.py", line 554, in _run_script
    exec(code, module.__dict__)
  File "stapp.py", line 11, in <module>
    sql_query, results = bq_qna(question)
NameError: name 'bq_qna' is not defined


In [46]:
bq_qna('which state has the highest value')



> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Use the following format:
    Question: "Question here"
    SQLQuery: "SQL Query to run"
    SQLResult: "Result of the SQLQuery"
    Answer:

('District of Columbia',
 {'input': 'You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.\n    Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.\n    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\n    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n    Use the following format:\n    Question: "Question here"\n    SQLQuery: "SQL Query to run"\n    SQLResult: "Result of the SQLQ

In [47]:
bq_qna('which state has the lowest value')



> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Use the following format:
    Question: "Question here"
    SQLQuery: "SQL Query to run"
    SQLResult: "Result of the SQLQuery"
    Answer:

('Idaho',
 {'input': 'You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.\n    Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.\n    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\n    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n    Use the following format:\n    Question: "Question here"\n    SQLQuery: "SQL Query to run"\n    SQLResult: "Result of the SQLQuery"\n    Answ

In [48]:
bq_qna('How many subpopulation categories were present in the table')



> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Use the following format:
    Question: "Question here"
    SQLQuery: "SQL Query to run"
    SQLResult: "Result of the SQLQuery"
    Answer:

('15',
 {'input': 'You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.\n    Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.\n    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\n    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n    Use the following format:\n    Question: "Question here"\n    SQLQuery: "SQL Query to run"\n    SQLResult: "Result of the SQLQuery"\n    Answer:

In [49]:
bq_qna('Name subpopulation categories were present in the table')



> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Use the following format:
    Question: "Question here"
    SQLQuery: "SQL Query to run"
    SQLResult: "Result of the SQLQuery"
    Answer:

('Asian/Pacific Islander, Black/African American, Other Race, American Indian/Alaska Native, Hispanic, Multiracial, Non-Metropolitan Area, White, College Grad, Female, Metropolitan Area, Male, Some College, None, Less Than High School, High School Grad',
 {'input': 'You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.\n    Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.\n    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\n    Pay attention to use only the column names you can see in the tables below. Be careful 

In [51]:
bq_qna('what is the value present in male and female for Washington')



> Entering new  chain...
You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Use the following format:
    Question: "Question here"
    SQLQuery: "SQL Query to run"
    SQLResult: "Result of the SQLQuery"
    Answer:

('5.0',
 {'input': 'You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.\n    Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.\n    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\n    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n    Use the following format:\n    Question: "Question here"\n    SQLQuery: "SQL Query to run"\n    SQLResult: "Result of the SQLQuery"\n    Answer